<a href="https://colab.research.google.com/github/PhiCtl/Flower_detection/blob/main/Object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import numpy as np
import torch
import torch.utils.data
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.transforms as T
import cv2
import pandas as pd

# ref https://github.com/pytorch/vision/blob/master/references/detection/
# ref https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

################################################################################
# GETTING STARTED
################################################################################

MEAN_Imagenet = [0.485, 0.456, 0.406]
STD_Imagenet = [0.229, 0.224, 0.225]

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


#### Utils

In [69]:
# defined in utils OK
def label_reader(json_file):
  data = pd.read_json(json_file)
  bboxes = {}
  for _, d in data.iterrows():
    name = d['External ID']
    labels = d['Label']['objects']
    pix = []
    for l in labels:
      b = l['bbox']
      pix_min = (b['top'],b['left'])
      pix_max = (b['top']+b['height'], b['left']+b['width'])
      pix.append([pix_min, pix_max])
    bboxes[name] = pix
  return bboxes

In [ ]:
# defined in utils

def get_transformed(train): # TODO: to test
  transforms = []
  # converts the image into a PyTorch Tensor
  transforms.append(T.ToTensor())
  # image scaling and normalization
  transforms.append(T.Normalize(mean = MEAN_Imagenet, std = STD_Imagenet))
  if train:
      # during training, randomly flip the training images
      # and ground-truth for data augmentation
      #TODO: investigate more data augmentation
      transforms.append(T.RandomHorizontalFlip(0.5))
      transforms.append(T.RandomRotation(degrees=90))
  return T.Compose(transforms)
  
# TODO define metrics to evaluate our model
def evaluate_model(model, test):
  model.eval()
  predictions = model(test['image'],test['target'])

# TODO : check if losses are correct, evaluate on training set
def train_model(model, train, num_epochs = 100):
  
  # set model to training mode
  model.train()

  # construct an optimizer
  params = [p for p in model.parameters() if p.requires_grad]
  optimizer = torch.optim.Adam(params, lr=0.005,
                        momentum=0.9, weight_decay=0.0005)
      
  # train over epochs
  for e in range(num_epochs):

    tot_loss = 0
    for d in train:
      # return losses NOT SURE TODO: check
      loss_dict = model(d['image'], d['target'])
      losses = sum(loss for loss in loss_dict.values())
      tot_loss += losses
      # update the learning rate
      optimizer.zero_grad()
      losses.backward()
      optimizer.step()

    # TODO evaluate here
    model.eval()
    predictions = model(train['image'],train['target'])
    if e % 10 == 0:
      print("Epochs {}/{}: training liss = {}, ".format(e, num_epochs))

## Creating a customed data set

In [ ]:
# defined in classes
class FlowerDetectionDataset(torch.utils.data.Dataset):

  def __init__(self, root, json_file, transforms=None):
    self.root = root
    self.transforms = transforms
    # load all image files, sorting them to
    # ensure that they are aligned
    self.imgs = list(sorted(os.listdir(os.path.join(root, "flowers"))))
    self.labels = label_reader(json_file)                                    

   def __len__(self):
        return len(self.imgs)
  
   def __getitem__(self, idx):

     # load images and bounding boxes
     img_path = os.path.join(self.root, "flowers", self.imgs[idx])
     img = cv2.imread(img_path)
     bboxes = self.labels[self.imgs[idx]]

     # there is only one class
     labels = torch.ones((num_objs,), dtype=torch.int64)
     
     # prepare sample
     bboxes = torch.as_tensor(bboxes, dtype=torch.float32)
     image_id = torch.tensor([idx])
     target = {}
     target["boxes"] = bboxes
     target["labels"] = labels
     target["image_id"] = image_id

     if self.transforms is not None:
         img = self.transforms(img)

     sample["image"], sample["target"] = img, target
     return sample


## Creating a customed model

In [ ]:
#define a model class
class myModel(torch.nn.Module):

  def __init__(self, model_type = 'fasterrcnn_resnet50_fpn', num_classes = 2):
    super().__init__()
    if model_type == '':
      self.model_ = #fill in
    else:
      self.model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
      in_features = self.model.roi_heads.box_predictor.cls_score.in_features
      self.model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

  def forward(self,x):
    return self.model(x)


# Defining our model
We will be using Faster R-CNN. Faster R-CNN is a model that predicts both bounding boxes and class scores for potential objects in the image.

In [ ]:

################################################################################
# LOAD DATASET
################################################################################
# TODO use our dataset
#dataset = 
#dataset_test = 

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

################################################################################
# TRAINING
################################################################################
# define model
# TODO

# move model to device
model.to(device)